In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor
import holidays

Load and Clean Data

In [2]:
# --- Load and Prepare Data ---
df = pd.read_csv("D:/Study/Hospital/HospitalData_Sam.csv")  # replace with your filename
df.columns = df.columns.str.strip()
df.head()


,Date,Day Of Week,Hour,No_of_arrivals
0,01/04/2022,Friday,00:00 - 00:59,5
1,01/04/2022,Friday,01:00 - 01:59,7
2,01/04/2022,Friday,02:00 - 02:59,3
3,01/04/2022,Friday,03:00 - 03:59,4
4,01/04/2022,Friday,04:00 - 04:59,5


Remove Outliers

In [3]:
# --- Step 1: Calculate IQR ---
Q1 = df['No_of_arrivals'].quantile(0.25)
Q3 = df['No_of_arrivals'].quantile(0.75)
IQR = Q3 - Q1

# Define bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# --- Step 2: Identify Outliers ---
outliers = df[(df['No_of_arrivals'] < lower_bound) | (df['No_of_arrivals'] > upper_bound)]
num_outliers = outliers.shape[0]
print(f"Number of outliers detected: {num_outliers}")

# --- Step 3: Remove Outliers ---
df = df[(df['No_of_arrivals'] >= lower_bound) & (df['No_of_arrivals'] <= upper_bound)]

Number of outliers detected: 32


Get March Features

In [4]:
march_mask = (df['Date'] >= '2025-03-01') & (df['Date'] < '2025-04-01')
df_march = df[march_mask].copy()

In [5]:
# --- Step 2: Convert date and extract hour ---
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Hour'] = df['Hour'].str.extract(r'^(\d{2}):')[0].astype(int)
# --- Step 3: Create Datetime column ---
df['Datetime'] = df['Date'] + pd.to_timedelta(df['Hour'], unit='h')
# --- Step 4: Remove March 2025 ---
# df = df[~((df['Datetime'].dt.year == 2025) & (df['Datetime'].dt.month == 3))]

# --- Step 5: Sort data by time ---
df = df.sort_values('Datetime')

In [6]:
df.head()

,Date,Day Of Week,Hour,No_of_arrivals,Datetime
0,2022-04-01,Friday,0,5,2022-04-01 00:00:00
1,2022-04-01,Friday,1,7,2022-04-01 01:00:00
2,2022-04-01,Friday,2,3,2022-04-01 02:00:00
3,2022-04-01,Friday,3,4,2022-04-01 03:00:00
4,2022-04-01,Friday,4,5,2022-04-01 04:00:00


Feature Engineering

In [7]:
# --- Step 6: Feature Engineering ---
df['Weekday'] = df['Datetime'].dt.weekday
df['Month'] = df['Datetime'].dt.month
df['Year'] = df['Datetime'].dt.year
df['Day'] = df['Datetime'].dt.day



In [8]:
# --- Step 7: Cyclical Encoding ---
df['Hour_sin'] = np.sin(2 * np.pi * df['Hour'] / 24)
df['Hour_cos'] = np.cos(2 * np.pi * df['Hour'] / 24)
df['Weekday_sin'] = np.sin(2 * np.pi * df['Weekday'] / 7)
df['Weekday_cos'] = np.cos(2 * np.pi * df['Weekday'] / 7)
df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
# Add rolling_hour_mean: average arrivals per hour over past 7 same-hours
df['rolling_weekday_hour_mean'] = (
    df.groupby(['Weekday', 'Hour'])['No_of_arrivals']
    .transform(lambda x: x.shift(1).rolling(4).mean())
)
df['lag_7d_same_hour'] = df['No_of_arrivals'].shift(24 * 7)

# Create a temporary key without adding it to df
day_hour_key = df['Day'].astype(str) + '_' + df['Hour'].astype(str)
# Calculate the average directly
day_hour_avg_map = df.groupby(day_hour_key)['No_of_arrivals'].mean()
# Assign day_hour_avg using .map(), no merge, no duplicate columns
df['day_hour_avg'] = day_hour_key.map(day_hour_avg_map)

# Create UK holiday calendar (covers England, Wales, Scotland, and Northern Ireland)
uk_holidays = holidays.UnitedKingdom()  # or use holidays.UK()
# If your data has a datetime column:
df['is_holiday'] = df['Date'].isin(uk_holidays).astype(int)
#Mark weekends as well
df['is_weekend'] = df['Weekday'].isin([5, 6]).astype(int)  # Saturday=5, Sunday=6

day_ohe = pd.get_dummies(df['Day'], prefix='day')
df = pd.concat([df, day_ohe], axis=1)
df.drop(columns=['Day'], inplace=True)

# ---  Step 8: Drop Redundant Columns ---
df.drop(columns=['Date'], inplace=True)


In [9]:
df.tail()

,Day Of Week,Hour,No_of_arrivals,Datetime,Weekday,Month,Year,Hour_sin,Hour_cos,Weekday_sin,...,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31
26228,Monday,19,19,2025-03-31 19:00:00,0,3,2025,-0.965926,0.258819,0.0,...,False,False,False,False,False,False,False,False,False,True
26229,Monday,20,16,2025-03-31 20:00:00,0,3,2025,-0.866025,0.500000,0.0,...,False,False,False,False,False,False,False,False,False,True
26230,Monday,21,16,2025-03-31 21:00:00,0,3,2025,-0.707107,0.707107,0.0,...,False,False,False,False,False,False,False,False,False,True
26231,Monday,22,9,2025-03-31 22:00:00,0,3,2025,-0.500000,0.866025,0.0,...,False,False,False,False,False,False,False,False,False,True
26232,Monday,23,5,2025-03-31 23:00:00,0,3,2025,-0.258819,0.965926,0.0,...,False,False,False,False,False,False,False,False,False,True


In [10]:
# --- Lag and rolling features ---
df['lag_1'] = df['No_of_arrivals'].shift(1)
df['lag_24'] = df['No_of_arrivals'].shift(24)
df['lag_168'] = df['No_of_arrivals'].shift(168)
df['rolling_mean_3'] = df['No_of_arrivals'].shift(1).rolling(3).mean()
df['rolling_std_24'] = df['No_of_arrivals'].shift(1).rolling(24).std()

In [11]:
# --- Drop rows with NaNs (from shift/rolling) ---
df.dropna(inplace=True)

In [12]:
df.columns

Index(['Day Of Week', 'Hour', 'No_of_arrivals', 'Datetime', 'Weekday', 'Month',
       'Year', 'Hour_sin', 'Hour_cos', 'Weekday_sin', 'Weekday_cos',
       'Month_sin', 'Month_cos', 'rolling_weekday_hour_mean',
       'lag_7d_same_hour', 'day_hour_avg', 'is_holiday', 'is_weekend', 'day_1',
       'day_2', 'day_3', 'day_4', 'day_5', 'day_6', 'day_7', 'day_8', 'day_9',
       'day_10', 'day_11', 'day_12', 'day_13', 'day_14', 'day_15', 'day_16',
       'day_17', 'day_18', 'day_19', 'day_20', 'day_21', 'day_22', 'day_23',
       'day_24', 'day_25', 'day_26', 'day_27', 'day_28', 'day_29', 'day_30',
       'day_31', 'lag_1', 'lag_24', 'lag_168', 'rolling_mean_3',
       'rolling_std_24'],
      dtype='object')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25529 entries, 669 to 26232
Data columns (total 54 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Day Of Week                25529 non-null  object        
 1   Hour                       25529 non-null  int32         
 2   No_of_arrivals             25529 non-null  int64         
 3   Datetime                   25529 non-null  datetime64[ns]
 4   Weekday                    25529 non-null  int32         
 5   Month                      25529 non-null  int32         
 6   Year                       25529 non-null  int32         
 7   Hour_sin                   25529 non-null  float64       
 8   Hour_cos                   25529 non-null  float64       
 9   Weekday_sin                25529 non-null  float64       
 10  Weekday_cos                25529 non-null  float64       
 11  Month_sin                  25529 non-null  float64       
 12  Month_c

In [14]:
# --- Step 9: Define features and target ---
# Define base features
base_features = [
    'Hour_sin', 'Hour_cos',
    'Weekday_sin', 'Weekday_cos',
    'Month_sin', 'Month_cos',
    'Year',
    'lag_1', 'lag_24', 'lag_168',
    'rolling_mean_3', 'rolling_std_24',
    'rolling_weekday_hour_mean', 'lag_7d_same_hour',
    'day_hour_avg', 'is_holiday','is_weekend'
]

# Add day_ohe columns
day_columns = [col for col in df.columns if col.startswith('day_') and col != 'day_hour_avg']

features = base_features + day_columns
X = df[features]
y = df['No_of_arrivals']

In [15]:
X.columns

Index(['Hour_sin', 'Hour_cos', 'Weekday_sin', 'Weekday_cos', 'Month_sin',
       'Month_cos', 'Year', 'lag_1', 'lag_24', 'lag_168', 'rolling_mean_3',
       'rolling_std_24', 'rolling_weekday_hour_mean', 'lag_7d_same_hour',
       'day_hour_avg', 'is_holiday', 'is_weekend', 'day_1', 'day_2', 'day_3',
       'day_4', 'day_5', 'day_6', 'day_7', 'day_8', 'day_9', 'day_10',
       'day_11', 'day_12', 'day_13', 'day_14', 'day_15', 'day_16', 'day_17',
       'day_18', 'day_19', 'day_20', 'day_21', 'day_22', 'day_23', 'day_24',
       'day_25', 'day_26', 'day_27', 'day_28', 'day_29', 'day_30', 'day_31'],
      dtype='object')

80%-10% Time Based Split (29 Train Months Out of 35 Months total)

In [16]:
# --- 80-20 Time-Based Split (28 train months out of 35) ---
train_cutoff_date = pd.to_datetime('2024-09-01')
data_cutoff_date = pd.to_datetime('2025-03-01')
X_train = X[df['Datetime'] < train_cutoff_date]
y_train = y[df['Datetime'] < train_cutoff_date]
X_test = X[(df['Datetime'] >= train_cutoff_date) & (df['Datetime']<data_cutoff_date)]
y_test = y[(df['Datetime'] >= train_cutoff_date) & (df['Datetime']<data_cutoff_date)]

Time Series Cross-Validation

In [17]:
# --- Time series cross-validation ---
tscv = TimeSeriesSplit(n_splits=3)

In [18]:
X_train[:5]

,Hour_sin,Hour_cos,Weekday_sin,Weekday_cos,Month_sin,Month_cos,Year,lag_1,lag_24,lag_168,...,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31
669,0.000000,1.000000,-0.433884,-0.900969,0.866025,-0.5,2022,6.0,6.0,8.0,...,False,False,False,False,False,False,False,True,False,False
670,0.258819,0.965926,-0.433884,-0.900969,0.866025,-0.5,2022,5.0,4.0,1.0,...,False,False,False,False,False,False,False,True,False,False
671,0.500000,0.866025,-0.433884,-0.900969,0.866025,-0.5,2022,5.0,4.0,3.0,...,False,False,False,False,False,False,False,True,False,False
672,0.707107,0.707107,-0.433884,-0.900969,0.866025,-0.5,2022,5.0,2.0,5.0,...,False,False,False,False,False,False,False,True,False,False
673,0.866025,0.500000,-0.433884,-0.900969,0.866025,-0.5,2022,2.0,4.0,3.0,...,False,False,False,False,False,False,False,True,False,False


In [19]:
X_train.columns

Index(['Hour_sin', 'Hour_cos', 'Weekday_sin', 'Weekday_cos', 'Month_sin',
       'Month_cos', 'Year', 'lag_1', 'lag_24', 'lag_168', 'rolling_mean_3',
       'rolling_std_24', 'rolling_weekday_hour_mean', 'lag_7d_same_hour',
       'day_hour_avg', 'is_holiday', 'is_weekend', 'day_1', 'day_2', 'day_3',
       'day_4', 'day_5', 'day_6', 'day_7', 'day_8', 'day_9', 'day_10',
       'day_11', 'day_12', 'day_13', 'day_14', 'day_15', 'day_16', 'day_17',
       'day_18', 'day_19', 'day_20', 'day_21', 'day_22', 'day_23', 'day_24',
       'day_25', 'day_26', 'day_27', 'day_28', 'day_29', 'day_30', 'day_31'],
      dtype='object')

In [20]:
print(df.columns[df.columns.duplicated()])

Index([], dtype='object')


Model Building

XGB Normal Deep Grid

In [21]:
# ---- 2. XGBOOST ----
xgb_deep_grid = {
    'n_estimators': [300, 500],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.005, 0.01, 0.05],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0],
    'min_child_weight': [1, 3],
    'reg_alpha': [0, 0.1],
    'reg_lambda': [1, 2],
    'gamma': [0, 1],
}
xgb_model = XGBRegressor(random_state=42, verbosity=0)
xgb_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=xgb_deep_grid,
    n_iter=25,
    cv=tscv,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

xgb_search.fit(X_train, y_train)
xgb_best = xgb_search.best_estimator_

# --- Evaluate XGB ---
xgb_preds = xgb_best.predict(X_test)
print("Tuned XGBoost:")
print(f"  MAE  = {mean_absolute_error(y_test, xgb_preds):.2f}")
print(f"  RMSE = {np.sqrt(mean_squared_error(y_test, xgb_preds)):.2f}")

Fitting 3 folds for each of 25 candidates, totalling 75 fits
Tuned XGBoost:
  MAE  = 2.49
  RMSE = 3.23


### Inference

Prepare Prediction for March

In [31]:
import pandas as pd
import numpy as np

# Step 1: Load and preprocess
df_full = pd.read_csv("D:/Study/Hospital/HospitalData_Sam.csv")
df_full.columns = df_full.columns.str.strip()
df_full['Date'] = pd.to_datetime(df_full['Date'], format='%d/%m/%Y')
df_full['Hour'] = df_full['Hour'].str.extract(r'^(\d{2}):')[0].astype(int)
df_full['Datetime'] = df_full['Date'] + pd.to_timedelta(df_full['Hour'], unit='h')
df_full.sort_values('Datetime', inplace=True)

# Step 2: Basic feature extraction
df_full['No_of_arrivals'] = df_full['No_of_arrivals'].astype(float)
df_full['Weekday'] = df_full['Datetime'].dt.weekday
df_full['Month'] = df_full['Datetime'].dt.month
df_full['Year'] = df_full['Datetime'].dt.year
df_full['Day'] = df_full['Datetime'].dt.day  # Keep temporarily for day_hour_avg
# If your data has a datetime column:
df_full['is_holiday'] = df_full['Date'].isin(uk_holidays).astype(int)
#Mark weekends as well
df_full['is_weekend'] = df_full['Weekday'].isin([5, 6]).astype(int)  # Saturday=5, Sunday=6

# Step 3: Cyclical features
df_full['Hour_sin'] = np.sin(2 * np.pi * df_full['Hour'] / 24)
df_full['Hour_cos'] = np.cos(2 * np.pi * df_full['Hour'] / 24)
df_full['Weekday_sin'] = np.sin(2 * np.pi * df_full['Weekday'] / 7)
df_full['Weekday_cos'] = np.cos(2 * np.pi * df_full['Weekday'] / 7)
df_full['Month_sin'] = np.sin(2 * np.pi * df_full['Month'] / 12)
df_full['Month_cos'] = np.cos(2 * np.pi * df_full['Month'] / 12)

# Step 4: Lag and rolling features
df_full['lag_1'] = df_full['No_of_arrivals'].shift(1)
df_full['lag_24'] = df_full['No_of_arrivals'].shift(24)
df_full['lag_168'] = df_full['No_of_arrivals'].shift(168)
df_full['rolling_mean_3'] = df_full['No_of_arrivals'].shift(1).rolling(3).mean()
df_full['rolling_std_24'] = df_full['No_of_arrivals'].shift(1).rolling(24).std()
df_full['rolling_weekday_hour_mean'] = (
    df_full.groupby(['Weekday', 'Hour'])['No_of_arrivals']
    .transform(lambda x: x.shift(1).rolling(4).mean())
)
df_full['lag_7d_same_hour'] = df_full['No_of_arrivals'].shift(24 * 7)

# Step 5: Add day_hour_avg (before one-hot encoding)
day_hour_key = df_full['Day'].astype(str) + '_' + df_full['Hour'].astype(str)
day_hour_avg_map = df_full.groupby(day_hour_key)['No_of_arrivals'].mean()
df_full['day_hour_avg'] = day_hour_key.map(day_hour_avg_map)

# Step 6: One-hot encode Day of Month
df_full = pd.get_dummies(df_full, columns=['Day'], prefix='day')


# Step 7: Drop NA values from lag/rolling
df_full.dropna(inplace=True)

# Step 8: Filter for March 2025
march_mask = (df_full['Datetime'].dt.year == 2025) & (df_full['Datetime'].dt.month == 3)
df_march = df_full.loc[march_mask].copy()

# Step 9: Feature selection
day_cols = [col for col in df_full.columns if col.startswith('day_') and col != 'day_hour_avg']
features = [
    'Hour_sin', 'Hour_cos',
    'Weekday_sin', 'Weekday_cos',
    'Month_sin', 'Month_cos',
    'Year',
    'lag_1', 'lag_24', 'lag_168',
    'rolling_mean_3', 'rolling_std_24',
    'rolling_weekday_hour_mean', 'lag_7d_same_hour',
    'day_hour_avg', 'is_holiday','is_weekend'
] + day_cols

X_march = df_march[features]
# Step 10: Predict arrivals
expected_features = list(xgb_best.feature_names_in_)  
missing_features = set(expected_features) - set(X_march.columns)
if missing_features:
    raise ValueError(f"Missing features in X_march: {missing_features}")
# Select and reorder columns safely
X_march = X_march[expected_features].astype(float)
# X_march.info()
# Predict
df_march['Predicted Arrival'] = np.clip(np.ceil(xgb_best.predict(X_march)).astype(int), 0, None)

# Step 11: Format for export
df_march['Date'] = df_march['Datetime'].dt.strftime('%d/%m/%Y')
df_march['Hour'] = df_march['Hour'].apply(lambda h: f"{h:02d}:00 - {h:02d}:59")
df_march['Day Of Week'] = df_march['Datetime'].dt.day_name()

result = df_march[['Date', 'Day Of Week', 'Hour', 'No_of_arrivals', 'Predicted Arrival']].copy()

# Step 12: Save
result.to_excel("D:/Study/Hospital/March_Arrival_Predictions.xlsx", index=False)
print("✅ Saved to March_Arrival_Predictions.xlsx")


✅ Saved to March_Arrival_Predictions.xlsx


Future Prediction

In [36]:
def generate_future_predictions(model, df, start_date, end_date):
    import pandas as pd
    import numpy as np

    # Step 1: Create future hourly datetime range
    future_dt_range = pd.date_range(start=start_date, end=end_date, freq='h')
    future_df = pd.DataFrame({'Datetime': future_dt_range})
    future_df['Date'] = future_df['Datetime'].dt.date
    future_df['Hour'] = future_df['Datetime'].dt.hour
    future_df['Weekday'] = future_df['Datetime'].dt.weekday
    future_df['Month'] = future_df['Datetime'].dt.month
    future_df['Year'] = future_df['Datetime'].dt.year
    future_df['Day'] = future_df['Datetime'].dt.day  # Needed for day_hour_avg and OHE
    
    # If your data has a datetime column:
    future_df['is_holiday'] = future_df['Date'].isin(uk_holidays).astype(int)
    #Mark weekends as well
    future_df['is_weekend'] = future_df['Weekday'].isin([5, 6]).astype(int)  # Saturday=5, Sunday=6

    # Step 2: Cyclical encodings
    future_df['Hour_sin'] = np.sin(2 * np.pi * future_df['Hour'] / 24)
    future_df['Hour_cos'] = np.cos(2 * np.pi * future_df['Hour'] / 24)
    future_df['Weekday_sin'] = np.sin(2 * np.pi * future_df['Weekday'] / 7)
    future_df['Weekday_cos'] = np.cos(2 * np.pi * future_df['Weekday'] / 7)
    future_df['Month_sin'] = np.sin(2 * np.pi * future_df['Month'] / 12)
    future_df['Month_cos'] = np.cos(2 * np.pi * future_df['Month'] / 12)

    # Step 3: Compute day_hour_avg from training history
    df['Day'] = df['Datetime'].dt.day
    df['Hour'] = df['Datetime'].dt.hour
    day_hour_key = df['Day'].astype(str) + '_' + df['Hour'].astype(str)
    day_hour_avg_map = df.groupby(day_hour_key)['No_of_arrivals'].mean()
    future_key = future_df['Day'].astype(str) + '_' + future_df['Hour'].astype(str)
    future_df['day_hour_avg'] = future_key.map(day_hour_avg_map)

    # Step 4: One-hot encode Day (after using it for day_hour_avg)
    future_df = pd.get_dummies(future_df, columns=['Day'], prefix='day')
    day_cols = [col for col in future_df.columns if col.startswith('day_')]
    future_df[day_cols] = future_df[day_cols].astype(int)

    # Step 5: Add lag/rolling values using latest history
    history = df[['Datetime', 'No_of_arrivals']].copy().set_index('Datetime').asfreq('h')
    full = pd.concat([history, future_df.set_index('Datetime')], axis=1)
    full['No_of_arrivals'] = full['No_of_arrivals'].ffill()

    full['lag_1'] = full['No_of_arrivals'].shift(1)
    full['lag_24'] = full['No_of_arrivals'].shift(24)
    full['lag_168'] = full['No_of_arrivals'].shift(168)
    full['lag_7d_same_hour'] = full['No_of_arrivals'].shift(24 * 7)
    full['rolling_mean_3'] = full['No_of_arrivals'].shift(1).rolling(3).mean()
    full['rolling_std_24'] = full['No_of_arrivals'].shift(1).rolling(24).std()

    # Step 6: rolling_weekday_hour_mean from training history
    df['Weekday'] = df['Datetime'].dt.weekday
    df['Hour'] = df['Datetime'].dt.hour
    weekday_hour_avg = (
        df.groupby(['Weekday', 'Hour'])['No_of_arrivals']
        .apply(lambda x: x.shift(1).rolling(4).mean())
        .reset_index(name='rolling_weekday_hour_mean')
    )
    df_temp = df.copy().reset_index(drop=True)
    df_temp['rolling_weekday_hour_mean'] = weekday_hour_avg['rolling_weekday_hour_mean']
    weekday_hour_lookup = (
        df_temp.groupby(['Weekday', 'Hour'])['rolling_weekday_hour_mean']
        .mean()
        .reset_index()
    )

    # Step 7: Merge back into full for future
    future_mean_lookup = future_df[['Datetime', 'Weekday', 'Hour']].copy()
    future_mean_lookup = future_mean_lookup.merge(
        weekday_hour_lookup, on=['Weekday', 'Hour'], how='left'
    ).set_index('Datetime')
    full['rolling_weekday_hour_mean'] = future_mean_lookup['rolling_weekday_hour_mean']

    # Step 8: Filter only future rows
    future_features = full.loc[future_dt_range].copy()
    future_features.dropna(inplace=True)

    # Step 9: Feature selection
    feature_cols = [
                        'Hour_sin', 'Hour_cos',
                        'Weekday_sin', 'Weekday_cos',
                        'Month_sin', 'Month_cos',
                        'Year',
                        'lag_1', 'lag_24', 'lag_168',
                        'rolling_mean_3', 'rolling_std_24',
                        'rolling_weekday_hour_mean', 'lag_7d_same_hour',
                        'day_hour_avg', 'is_holiday','is_weekend'
                    ] + [col for col in future_features.columns if col.startswith('day_') and col != 'day_hour_avg']

    # Step 10: Predict
    X_future = future_features[feature_cols]
    future_features['Predicted Arrival'] = np.clip(np.ceil(model.predict(X_future)).astype(int), 0, None)

    # Step 11: Final formatting
    future_features['Date'] = future_features.index.date
    future_features['Hour'] = future_features.index.hour
    future_features['Weekday'] = pd.to_datetime(future_features.index).day_name()

    return future_features[['Date', 'Weekday', 'Hour', 'Predicted Arrival']].reset_index(drop=True)


In [37]:
# Predict hourly arrivals from April 1 to April 3, 2025
future_pred_df = generate_future_predictions(xgb_best, df, '2025-07-01', '2026-08-31 23:00')

# Show or save
print(future_pred_df.head())
future_pred_df.to_excel("Future_Arrival_Predictions.xlsx", index=False)

         Date  Weekday  Hour  Predicted Arrival
0  2025-07-01  Tuesday     0                  7
1  2025-07-01  Tuesday     1                  7
2  2025-07-01  Tuesday     2                  7
3  2025-07-01  Tuesday     3                  6
4  2025-07-01  Tuesday     4                  6


In [ ]:
import matplotlib.pyplot as plt

# Load the dataset
df.columns = df.columns.str.strip()

# Plot boxplot for outlier inspection
plt.figure(figsize=(12, 6))
plt.boxplot(df['No_of_arrivals'], vert=False)
plt.title('Box Plot of Hourly Number of Arrivals')
plt.xlabel('No of Arrivals')
plt.grid(True)
plt.tight_layout()
plt.show()